## CSCI-SHU 210 Data Structures

### Recitation 8 Linked Lists

March April 2022

Yisong Wang

# 1. Single Linked List
## Implement a SLL: 
- What methods do you need? 
- basic attributes
- basic methods

In [ ]:
class Empty(Exception):
    pass

class Node:
    __slots__ = '_element', '_next'

    def __init__(self, element, next):
        self._element = element # Polyterm(1,3), x^3
        self._next = next  # should be linked to a Node

In [ ]:
class SinglyLinkedList:
    def __init__(self):
        self._head = None
        self._tail = None
        self._size = 0
      
    def __len__(self):
        return self._size

    def is_empty(self):
        return self._size == 0
    
    def add_first(self,e):
        new_node = Node(e,self._head)
        # The new node, which will point to the original head
        if self.is_empty():
            self._tail = new_node
        
        self._head = new_node
        self._size += 1
        return new_node

    def del_first(self):
        if self.is_empty():
            raise Empty('Stack is empty')
        answer = self._head._element

        self._head = self._head._next
        self._size -= 1

        if self.is_empty():
            self._tail = None
        
        return answer

    def add_last(self, e):
        new_node = Node(e,None)
        if self.is_empty():
            self._head = new_node
        else:
            self._tail._next = new_node
        self._tail = new_node
        self._size += 1
        return new_node
    
    def __getitem__(self, k):
        if k < 0:
            k += self._size
        if not -1 < k < self._size:
            raise IndexError("Either empty list, or poor index")
        target = self._head
        for i in range(k):
            target = target._next
        return target._element

    def unordered_search(self,target):
        node = self._head
        while node is not None:
          if node._element == target:
            return True
          node = node._next
        return False
        # for node_elem in self:
        #   if node_elem == target:
        #     return True
        # return False

### Write the following later
    def del_and_reconnect(self, before_node):
        target = before_node._next._element
        before_node._next = before_node._next._next
        # print(before_node._element, before_node._next)
        self._size -= 1
        # Special case: when the target node is the actual tail node 
        if before_node._next == None:
          self._tail = before_node
        return target
    
    def insert_after(self, e, before_node):
        if self._tail == before_node:
          return self.add_last(e)
        new_node = Node(e,before_node._next)
        # The new node, which will point to the original head
        before_node._next = new_node
        self._size += 1
        return new_node
    
    def __str__(self):
        ret = []
        next_node = self._head
        while next_node:
            ret.append(str(next_node._element))
            next_node = next_node._next
        return str(ret)

In [ ]:
a = SinglyLinkedList()
a.add_first(1)
a.add_first(2)
for i in range(3,10):
    a.add_first(i*(-1)**(i%2))
print(a)
print("length is", len(a))
print(a[3])
a.add_last(200)
print(a.unordered_search(200))

c = a._head._next
a.del_and_reconnect(c)
print(a)
a.insert_after(100,c)
print(a)

['-9', '8', '-7', '6', '-5', '4', '-3', '2', '1']
length is 9
6
True
['-9', '8', '6', '-5', '4', '-3', '2', '1', '200']
['-9', '8', '100', '6', '-5', '4', '-3', '2', '1', '200']


# 2. Using SLL: Polynomials
**Problem**: Use SLL as a basic structure, to create a basic polynomial storage
Think about it:
- where to use SLL?
- how to store elements? What is the node?
- Could we evaluate?
- How to add polynomials

In [ ]:
class Polyterm:
    __slots__ = '_coef', '_pow'

    def __init__(self, coef, power):
        self._coef = coef
        self._pow = power

    def evaluate(self,x):
        return self._coef*(x**self._pow)

    def __lt__(self, other):
        return self._pow < other._pow

    def __gt__(self, other):
        return self._pow > other._pow

    def __eq__(self, other):
        return self._pow == other ._pow

    def is_valid(self):
        if self._coef == 0:
            print("zero coefficient")
            return None
        return self

    def collect(self, other):  # __iadd__(self, other)
        assert self == other
        self._coef += other._coef

    def __str__(self):
        return str(self._coef) if self._pow == 0 else ((str(self._coef) if self._coef != 1 else "" ) + "x" + ("^"+str(self._pow) if self._pow != 1 else ""))

In [ ]:
a = Polyterm(2,5)
b = Polyterm(-1,5)
a.collect(b)
print(a.is_valid())

x^5


In [ ]:
class Polynomial:
    def __init__(self):
        self._data = SinglyLinkedList()
        # Node.collect = lambda x, y: x._element.collect(y)
    
    def __len__(self):
        return len(self._data)

    def is_empty(self):
        return self._data.is_empty()
    
    def add_first(self, new_term):
        return self._data.add_first(new_term) if new_term.is_valid() else None

    def add_last(self,new_term):
        return self._data.add_last(new_term) if new_term.is_valid() else None

    def extend(self, other):
        if len(other)==0:
            return self
        walk = other._data._head
        while walk:
            self.add_last(walk._element)
            walk = walk._next
        return self
 
    def insert_term(self, new_term):
        if not new_term.is_valid():
            return None

        walk = self._data._head
        if self.is_empty()  or walk._element > new_term:
            return self.add_first(new_term)
        if walk._element == new_term:
            walk._element.collect(new_term)
            return self._data.del_first() if not walk._element.is_valid() else None

        while walk._next and walk._next._element < new_term :
            walk = walk._next

        if not walk._next:
            return self.add_last(new_term)

        if walk._next._element == new_term:
            walk._next._element.collect(new_term)
            return self._data.del_and_reconnect(walk) if not walk._next._element.is_valid() else None

        return self._data.insert_after(new_term,walk)

    def evaluate(self, x):
        ret = 0
        for term in self._data:
            ret += term.evaluate(x)
        return ret

    def __add__(self, other):
        new_poly = Polynomial()
        if len(other) == 0:
            return new_poly.extend(self)
        if len(self) == 0:
            return new_poly.extend(other)

        base = self._data._head
        target = other._data._head
        while base and target:
            if target._element > base._element:
                new_poly.add_last(base._element)
                base = base._next
            elif target._element < base._element:
                new_poly.add_last(target._element)
                target = target._next
            else: 
                temp_term = Polyterm(base._element._coef, base._element._pow)
                temp_term.collect(target._element)
                # print(temp_term)
                if temp_term.is_valid():
                  new_poly.add_last(temp_term)

                base = base._next
                target = target._next
        while base:
            new_poly.add_last(base._element)
            base = base._next
        while target:
            new_poly.add_last(target._element)
            target = target._next
        return new_poly
        
    def __iadd__(self, other):
        base = self._data._head
        target = other._data._head
        while target._next:
          self.insert_term(Polyterm(target._element._coef, target._element._pow))
          target = target._next
          # print(self)
          # print(target._element if target else None)
        self.add_last(target._element)
        # print(target._element)
        # print(self)
        return self

    def __str__(self):
        return str(self._data)

In [ ]:
a = Polynomial()
a.add_first(Polyterm(1,0))
a.add_last(Polyterm(1,10))
for i in range(2,14,2):
    a.insert_term(Polyterm(2,i))
print(a)
a.insert_term(Polyterm(-2,13))
print(a)
a.insert_term(Polyterm(2,13))
print(a)

a.insert_term(Polyterm(2,13))
print(a)

['1', '2x^2', '2x^4', '2x^6', '2x^8', '3x^10', '2x^12']
['1', '2x^2', '2x^4', '2x^6', '2x^8', '3x^10', '2x^12', '-2x^13']
zero coefficient
['1', '2x^2', '2x^4', '2x^6', '2x^8', '3x^10', '2x^12']
['1', '2x^2', '2x^4', '2x^6', '2x^8', '3x^10', '2x^12', '2x^13']


In [ ]:
a = Polynomial()
a.add_first(Polyterm(1,0))
a.add_last(Polyterm(1,10))
for i in range(2,14,2):
    a.insert_term(Polyterm(2,i))
print(a)
print(a.evaluate(1))

# a.insert_term(Polyterm(-2,4))
# print(a)

b = Polynomial()
b.extend(a)
print(b)

c = Polynomial()
for i in range(2,14):
    c.insert_term(Polyterm(-2,i))
print(c)

d = a + c
print('now, d=', d)
print('a is the same', a)
print('c is the same', c)

a += c
print('now, a=', a)
print('c is the same', c)

['1', '2x^2', '2x^4', '2x^6', '2x^8', '3x^10', '2x^12']
14
['1', '2x^2', '2x^4', '2x^6', '2x^8', '3x^10', '2x^12']
['-2x^2', '-2x^3', '-2x^4', '-2x^5', '-2x^6', '-2x^7', '-2x^8', '-2x^9', '-2x^10', '-2x^11', '-2x^12', '-2x^13']
zero coefficient
zero coefficient
zero coefficient
zero coefficient
zero coefficient
now, d= ['1', '-2x^3', '-2x^5', '-2x^7', '-2x^9', 'x^10', '-2x^11', '-2x^13']
a is the same ['1', '2x^2', '2x^4', '2x^6', '2x^8', '3x^10', '2x^12']
c is the same ['-2x^2', '-2x^3', '-2x^4', '-2x^5', '-2x^6', '-2x^7', '-2x^8', '-2x^9', '-2x^10', '-2x^11', '-2x^12', '-2x^13']
zero coefficient
zero coefficient
zero coefficient
zero coefficient
zero coefficient
now, a= ['1', '-2x^3', '-2x^5', '-2x^7', '-2x^9', 'x^10', '-2x^11', '-2x^13']
c is the same ['-2x^2', '-2x^3', '-2x^4', '-2x^5', '-2x^6', '-2x^7', '-2x^8', '-2x^9', '-2x^10', '-2x^11', '-2x^12', '-2x^13']


# 3. Doubly Linked List

- What methods do you need? 
- basic attributes
- basic methods

In [ ]:
class DoublyLinkedList:
    class _Node:
        __slots__ = '_element', '_prev','_next'

        def __init__(self, element, prev, next):
            self._element = element
            self._prev = prev
            self._next = next  # should be linked to a Node

        def get_succ(self):
            return self._next

        def get_pred(self):
            return self._prev

        def get_value(self):
            return self._element

    def __init__(self):
        self._head = self._Node(None, None, None)  # the sentinel
        self._tail = self._Node(None, self._head, None) # these two are not equal  the sentinel
        self._head._next = self._tail
        self._size = 0

    def __len__(self):
        return self._size

    def is_empty(self):
        return self._size == 0

    def first(self): 
        return self._head._next

    def last(self):
        return self._tail._prev

    def insert_between(self, e, pred, succ):
        new_node = self._Node(e, pred, succ)
        pred._next = new_node
        succ._prev = new_node
        self._size += 1
        return new_node

    def delete_node(self, target):
        target._prev._next = target._next
        target._next._prev = target._prev
        self._size -= 1
        answer = target._element
        target._prev = target._next = target._element = None
        return answer

    def add_first(self,e):
        return self.insert_between(e,self._head,self.first())

    def add_last(self,e):
        return self.insert_between(e,self.last(), self._tail)

    def del_first(self):
        if self.is_empty():
            raise Empty("DLL is empty")
        return self.delete_node(self.first())

    def del_last(self):
        if self.is_empty():
            raise Empty("DLL is empty")
        return self.delete_node(self.last())

    def search_node(self,e):
        walk = self.first()
        # Check the first condition before checking the second; i.e, the second would not raise error if the first is invalid
        while walk != self._tail and walk.get_value() != e:
            walk = walk.get_succ()
        if walk == self._tail:
            return None # False
        return walk # True

    def __str__(self):
        ret = ["head="]
        next_node = self.first()
        while next_node._next:
            ret.append(str(next_node._element)+"=")
            next_node = next_node._next
        ret.append("tail")
        return ''.join(ret)

In [ ]:
a = DoublyLinkedList()
for i in range(5):
    a.add_first(i)
    a.add_last(9-i)
a.insert_between(10, a.first()._next, a.first()._next._next)
print(a)


head=4=3=10=2=1=0=9=8=7=6=5=tail


# 4. Using DLL
**Problem** Do insertion sort on DLL
- recall how we do in list
- what behaviours do we need at minimum? Why DLL not SLL
- How to 'destroy the links' and 'reconnect'

In [ ]:
def insertion_sort(DLL):
    if len(DLL)>1:
        marker = DLL.first()
        while marker != DLL.last():
            pivot = marker.get_succ()
            value = pivot.get_value()
            if value > marker.get_value():
                marker = pivot
            else:
                walk = marker
                while walk != DLL.first() and walk.get_pred().get_value()>value:
                    walk = walk.get_pred()
                DLL.delete_node(pivot)
                DLL.insert_between(value, walk.get_pred(), walk)

In [ ]:
insertion_sort(a)
print(a)

head=0=1=2=3=4=5=6=7=8=9=10=tail


# 5. Using DLL
Suppose you have a favourite list: store webpages that you visit most often.
- store the element in the order of visiting freqeuncy
- implement using DLL: why DLL?
- access a webpage---what is a webpage Node?

In [ ]:
class Fav_item:
    __slots__ = '_value','_count'

    def __init__(self, e):
        self._value = e
        self._count = 0

    def __str__(self):
        return str((self._value, self._count))

In [ ]:
class FavDLL:
    def __init__(self):
        self._data = DoublyLinkedList()
        DoublyLinkedList._Node.get_value = lambda x: x._element._value

    def __len__(self):
        return len(self._data)

    def is_empty(self):
        return len(self._data) == 0

    # Locate the position instance of an element
    def _find_position(self, e):
        # change the method get_value() this will help the search method
        return self._data.search_node(e)
   
    def access(self,e):
        node_new = self._find_position(e)
        if not node_new:
            node_new = self._data.add_last(Fav_item(e))
        node_new._element._count += 1
        self._move_up(node_new)

    def _move_up(self, actual_node):
        if actual_node != self._data.first():
            cnt = actual_node._element._count
            walk = actual_node._prev
            if cnt > walk._element._count:
                while (walk != self._data.first()) and cnt > walk._prev._element._count:
                    walk = walk._prev
                self._data.insert_between(self._data.delete_node(actual_node), walk._prev, walk)
 
    def remove(self, e):
        node_e = self._find_position(e)
        if node_e:
            return self._data.delete_node(node_e)
        return None

    def top(self, k):
        if not 1 <= k <= len(self):
            raise ValueError('Illegal value for k')
        walk = self._data.first()
        # print(walk)
        for j in range(k):
            yield walk.get_value()
            walk = walk._next

    def __str__(self):
        return str(self._data)

In [ ]:
a = FavDLL()
a._data.add_last(Fav_item('b'))

a._data.first()._element._count+=1
print(a._data.first()._element)

for i in range(10):
    a.access('d')
    a.access('e')
    if i % 2 == 0:
        a.access('c')
a.access('d')
a.access('f')
a.access('g')
print(a)
a.remove('b')
print(a)
for i in a.top(2):
    print(i)

('b', 1)
head=('d', 11)=('e', 10)=('c', 5)=('b', 1)=('f', 1)=('g', 1)=tail
head=('d', 11)=('e', 10)=('c', 5)=('f', 1)=('g', 1)=tail
d
e
